In [ ]:
!sudo apt update
!sudo apt install rtl-sdr

In [ ]:
!pip install pyrtlsdr

In [ ]:
import socket
try:
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(5)
    s.connect(('xxx.xxx.xxx.xxx', 6xxx))
    print("✅ Connected successfully!")
    s.close()
except Exception as e:
    print(f"❌ Connection failed: {e}")

In [ ]:
# Cell 1: ติดตั้ง dependencies
!pip install numpy matplotlib -q

# Cell 2: Import libraries
import socket
import struct
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import time
from datetime import datetime

# Cell 3: RTLTCPClient Class
class RTLTCPClient:
    def __init__(self, hostname, port):
        self.hostname = hostname
        self.port = port
        self.sock = None

    def connect(self):
        """เชื่อมต่อกับ rtl-tcp server"""
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.settimeout(10)
        self.sock.connect((self.hostname, self.port))
        print(f"✅ Connected to {self.hostname}:{self.port}")

        # อ่าน dongle info (12 bytes แรก)
        info = self.sock.recv(12)
        print(f"Dongle info received: {len(info)} bytes")

    def send_command(self, cmd, param):
        """ส่งคำสั่งไปที่ rtl-tcp"""
        data = struct.pack('>BI', cmd, param)
        self.sock.send(data)

    def set_frequency(self, freq_hz):
        """ตั้งค่าความถี่ (Hz)"""
        self.send_command(0x01, int(freq_hz))
        print(f"📡 Set frequency: {freq_hz/1e6:.2f} MHz")

    def set_sample_rate(self, rate_hz):
        """ตั้งค่า sample rate (Hz)"""
        self.send_command(0x02, int(rate_hz))
        print(f"📊 Set sample rate: {rate_hz/1e6:.2f} MSps")

    def set_gain_mode(self, manual=False):
        """ตั้งค่า gain mode (0=auto, 1=manual)"""
        self.send_command(0x03, 1 if manual else 0)
        print(f"🎚️ Set gain mode: {'manual' if manual else 'auto'}")

    def set_gain(self, gain_tenth_db):
        """ตั้งค่า gain (หน่วย 0.1 dB)"""
        self.send_command(0x04, int(gain_tenth_db))
        print(f"📈 Set gain: {gain_tenth_db/10:.1f} dB")

    def read_samples(self, num_samples):
        """อ่าน IQ samples"""
        num_bytes = num_samples * 2
        data = b''

        while len(data) < num_bytes:
            chunk = self.sock.recv(min(8192, num_bytes - len(data)))
            if not chunk:
                break
            data += chunk

        # แปลง uint8 → float
        iq = np.frombuffer(data, dtype=np.uint8)
        iq = (iq - 127.5) / 127.5

        # แยก I และ Q
        i = iq[0::2]
        q = iq[1::2]

        # สร้าง complex samples
        samples = i + 1j * q

        return samples

    def close(self):
        """ปิดการเชื่อมต่อ"""
        if self.sock:
            self.sock.close()
            print("🔌 Connection closed")

# Cell 4: Setup และรัน Loop อัพเดททุก 5 วินาที
%matplotlib inline

# ===== ตั้งค่าที่นี่ =====
HOSTNAME = '167.172.7.22'
PORT = 6001
CENTER_FREQ = 185.36e6  # 185.36 MHz
SAMPLE_RATE = 2.048e6   # 2.048 MHz
NUM_SAMPLES = 128 * 1024  # จำนวน samples ที่จะอ่านแต่ละรอบ
UPDATE_INTERVAL = 3  # วินาที

print("🔧 Connecting to RTL-TCP server...")
sdr = RTLTCPClient(hostname=HOSTNAME, port=PORT)
sdr.connect()

# ตั้งค่า SDR
sdr.set_sample_rate(SAMPLE_RATE)
sdr.set_frequency(CENTER_FREQ)
sdr.set_gain_mode(manual=False)

print("\n✅ Setup complete!")
print(f"🔄 Will update every {UPDATE_INTERVAL} seconds")
print("⏹️  Stop the cell to terminate\n")

# สร้าง figure
fig = plt.figure(figsize=(15, 10))

try:
    iteration = 0
    while True:
        iteration += 1
        start_time = time.time()

        print(f"📡 Reading samples... (Iteration {iteration}) - {datetime.now().strftime('%H:%M:%S')}")
        samples = sdr.read_samples(NUM_SAMPLES)
        print(f"✅ Received {len(samples)} samples")

        # Clear figure
        fig.clear()

        # Plot 1: Time domain (แสดงแค่ 1000 samples แรก)
        ax1 = fig.add_subplot(3, 1, 1)
        ax1.plot(np.real(samples[:1000]), label='I', alpha=0.7, linewidth=0.8)
        ax1.plot(np.imag(samples[:1000]), label='Q', alpha=0.7, linewidth=0.8)
        ax1.set_xlabel('Sample')
        ax1.set_ylabel('Amplitude')
        ax1.set_title(f'Time Domain - IQ Samples (Updated: {datetime.now().strftime("%H:%M:%S")})')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # Plot 2: FFT / Spectrum
        ax2 = fig.add_subplot(3, 1, 2)
        fft_data = np.fft.fftshift(np.fft.fft(samples))
        fft_db = 20 * np.log10(np.abs(fft_data) + 1e-10)
        freqs = np.fft.fftshift(np.fft.fftfreq(len(samples), 1/SAMPLE_RATE))
        ax2.plot(freqs/1e6, fft_db, linewidth=0.8)
        ax2.set_xlabel('Frequency offset (MHz)')
        ax2.set_ylabel('Power (dB)')
        ax2.set_title(f'Spectrum at {CENTER_FREQ/1e6:.2f} MHz')
        ax2.grid(True, alpha=0.3)

        # หา peak
        peak_idx = np.argmax(fft_db)
        peak_freq = freqs[peak_idx]/1e6
        peak_power = fft_db[peak_idx]
        ax2.axvline(peak_freq, color='red', linestyle='--', alpha=0.5, linewidth=1)
        ax2.text(0.02, 0.98, f'Peak: {peak_freq:.3f} MHz offset @ {peak_power:.1f} dB',
                transform=ax2.transAxes,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

        # Plot 3: Spectrogram
        ax3 = fig.add_subplot(3, 1, 3)
        ax3.specgram(samples, NFFT=1024, Fs=SAMPLE_RATE/1e6, scale='dB', cmap='viridis')
        ax3.set_xlabel('Time (s)')
        ax3.set_ylabel('Frequency (MHz)')
        ax3.set_title('Spectrogram')

        plt.tight_layout()

        # Update display
        display.clear_output(wait=True)
        display.display(fig)

        # คำนวณเวลาที่ใช้ไป
        elapsed = time.time() - start_time
        print(f"⏱️  Processing time: {elapsed:.2f}s")

        # รอให้ครบ UPDATE_INTERVAL วินาที
        wait_time = UPDATE_INTERVAL - elapsed
        if wait_time > 0:
            print(f"💤 Waiting {wait_time:.1f}s until next update...\n")
            time.sleep(wait_time)
        else:
            print(f"⚠️  Processing took longer than update interval!\n")

except KeyboardInterrupt:
    print("\n⏹️  Stopped by user")
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()
finally:
    sdr.close()
    print("✅ Done!")